In [ ]:
import os
import shutil
import glob
import csv
import cv2
import scipy

from seg_functions_TC import*

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from PIL import Image, ImageDraw, ImageEnhance
from scipy.spatial import ConvexHull
from skimage import measure
import import_ipynb
import find_p2 as p2

In [ ]:
MAIN_PATH = r'D:\TeamChallengedataTUe\Team challenge 2021\Scoliose'
cso_file = os.path.join(MAIN_PATH,'Scoliose/1preopop.cso') 
nifti_file = os.path.join(MAIN_PATH,'Scoliose/1preop.nii')

ct_img = nib.load(nifti_file)
pixdim = find_pix_dim(ct_img)
ct_numpy_all = ct_img.get_fdata()


slice = 218
ct_numpy = ct_numpy_all[:,:,slice]
im_90 = np.rot90(ct_numpy)

In [ ]:
def filtered_images(ct_image_2d):
    
    HUbone = 200 #bone is around 300-400 HU (taking artefacts such as streaking- and cupping artefacts in account so decreasing to 200)
    ret,thresh = cv2.threshold(ct_image_2d,HUbone,3000,cv2.THRESH_BINARY)
    
    # om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt
    thres = thresh.astype('int16')
    bone_threshold_filtered_image = cv2.medianBlur(thres, 3)
    
    return bone_threshold_filtered_image

bone_im = filtered_images(im_90)


In [ ]:
lung_seg = -500
_,thresh_bone = cv2.threshold(im_90,lung_seg,10,cv2.THRESH_BINARY_INV)
# thresh = thresh.astype('int8')

#closing uitvoeren
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(thresh_bone, cv2.MORPH_CLOSE, kernel, iterations=3)
closing1 = closing.astype('uint8') # om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt

contours, hierarchy = cv2.findContours(closing1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
imgplot = plt.imshow(closing1, cmap='gray')
closing_norm = closing1/closing1.max()
cv2.drawContours(closing1, contours,2,(0,255,0))

h, w = closing1.shape[:2]
mask = np.zeros((h+2, w+2), np.uint8)
cv2.floodFill(closing1, mask, (0,0), 0)
cv2.floodFill(closing1, mask, (511,511), 0)
imgplot = plt.imshow(closing1, cmap='gray')


connectivity = 4
markers = cv2.connectedComponentsWithStats(closing1, connectivity, cv2.CV_32S)
num_labels = markers[0]
img = markers[1]
stats = markers[2]
centroi = markers[3]
# imgplot = plt.imshow(img, cmap='gray')
row_centr = np.sort([i for i in centroi[:,1] if i <= 400 and i > 100])

arr = np.nonzero(img)
l1 = [arr[0][0]],[arr[1][0]] #eerste punt (arr[rijnummer][kolomnummer]

# één masker verwijderen
img[img == img[l1]] = 0 # markers in markers die ongelijk zijn aan de waarde van het eerste punt 
arr2 = np.nonzero(img)
l2 = [arr2[0][0]],[arr2[1][0]] 




# lung0_mask = row_centr[0] == filtered_mask_area[0]



In [ ]:
imgplot = plt.imshow(im_90, cmap='gray')
plt.plot(l1[1], l1[0], marker='o', markersize=5, color="red")
plt.plot(l2[1], l2[0], marker='o', markersize=5, color="blue")


In [ ]:
fov_between_lungs = l1[1], l2[1]
fov_between_lungs = np.sort(fov_between_lungs, axis=None)
columns = np.prod(closing[1].shape)
mask = np.zeros(closing.shape)
mask[l1[0][0]-50:l1[0][0]+50,fov_between_lungs[0]:fov_between_lungs[1]]=1

middle_col = np.around((fov_between_lungs[0]+fov_between_lungs[1])/2)
# middle_row = np.around()
print(fov_between_lungs[0],fov_between_lungs[1])
fov_im90 = bone_im*mask

img = imgplot = plt.imshow(fov_im90, cmap='gray')


In [ ]:


# posterior middelpunt sternum bepalen:

#closing uitvoeren
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(fov_im90, cv2.MORPH_CLOSE, kernel, iterations=3)
closing = closing.astype('int8')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt



# Choose 4 or 8 for connectivity type
connectivity = 4  
masks = cv2.connectedComponentsWithStats(closing, connectivity, cv2.CV_32S)

num_labels = masks[0]
stats = masks[2]
centroids = masks[3]

for label in range(1,num_labels):
    blob_area = stats[label, cv2.CC_STAT_AREA]
    blob_width = stats[label, cv2.CC_STAT_WIDTH]
    blob_height = stats[label, cv2.CC_STAT_HEIGHT]

mask_area = stats[:,4]
filtered_mask_area = np.sort([x for x in mask_area if x <= 100000 and x > 150])
a = stats[:,4] == filtered_mask_area[0]
b=[i for i, x in enumerate(a) if x]
centroid_sternum = np.around(centroids[b])
# kleine correctiefactor ipv van de rand opzoeken (robuuster aangezien deze anders wervels detecteert)
lower_sternum = centroid_sternum[0][1]+10 






In [ ]:
# dit is de oude code die het posterior punt bepaald van p2
        # row_centroid_sternum = np.uint(centroid_sternum[0][1])
        # col_centroid_sternum = np.uint(centroid_sternum[0][0])
        # centroid_axis = closing[:,col_centroid_sternum]
        # centroid_axis_corr = centroid_axis[row_centroid_sternum:]

        # col_centroid_sternum = col_centroid_sternum
        # # find indices where the value changes.
        # index_backgroud_mask = np.where(centroid_axis_corr[:-1] != centroid_axis_corr[1:])[0] 
        # # add the value of the first index to the centroid row value
        # posterior_centroid_sternum_row = index_backgroud_mask[0]+row_centroid_sternum

In [ ]:
imgplot = plt.imshow(im_90, cmap='gray')
plt.plot(centroid_sternum[0][0], lower_sternum, marker='o', markersize=5, color="red")


###DE CODE HIERONDER IS NIET VAN TOEPASSING, IS ALLEEN VOOR DINGEN UIT TE PROBEREN

In [ ]:
HUbone = 200 #bone is around 300-400 HU (taking artefacts such as streaking- and cupping artefacts in account so decreasing to 200)
ret,thresh = cv2.threshold(im_90,HUbone,3000,cv2.THRESH_BINARY)

In [ ]:
thres = thresh.astype('int16')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt

bones_thresh_filt5 = cv2.medianBlur(thres, 5)
bones_thresh_filt3 = cv2.medianBlur(thres, 3)
fig, ax = plt.subplots(1, 2, figsize=(20, 20))
ax[0].imshow(bones_thresh_filt3, cmap='gray')
ax[0].set_title('median blur = 3')
ax[1].imshow(bones_thresh_filt5, cmap='gray')
ax[1].set_title('median blur = 5')

In [ ]:
# posterior middelpunt sternum bepalen:

#closing uitvoeren
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(bones_thresh_filt5, cv2.MORPH_CLOSE, kernel, iterations=3)
closing = closing.astype('int8')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt
imgplot = plt.imshow(closing, cmap='gray')


# Choose 4 or 8 for connectivity type
connectivity = 4  
masks = cv2.connectedComponentsWithStats(closing, connectivity, cv2.CV_32S)

num_labels = masks[0]
stats = masks[2]
centroids = masks[3]

for label in range(1,num_labels):
    blob_area = stats[label, cv2.CC_STAT_AREA]
    blob_width = stats[label, cv2.CC_STAT_WIDTH]
    blob_height = stats[label, cv2.CC_STAT_HEIGHT]

centroids_sorted = np.sort(centroids,1)

mask_area = stats[:,4]
filtered_mask_area = np.sort([x for x in mask_area if x <= 100000 and x > 250])
a = stats[:,4] == filtered_mask_area[0]
b=[i for i, x in enumerate(a) if x]
centroid_sternum = np.around(centroids[b])

# find lowest nonzero point in mask

    
# ind=np.argsort(stats[:,-1])
# stats[ind]
# stats = np.sort(stats[:,])    
# print(filtered_mask_area)
# print(centroid_sternum)
# print(centroid_sternum[0][0])
# print(centroids)

#ret,markers = cv2.connectedComponents(closing)


#find lowest point of mask in centroid
print(centroid_sternum[0][0])
r_centroid = centroid_sternum[0][0]
c_centroid = centroid_sternum[0][1]

# #plot result
# rot90_im = np.rot90(ct_numpy)
# imgplot = plt.imshow(im_90,cmap = 'gray')
# plt.plot(r_centroid, c_centroid, marker='o', markersize=5, color="green")




In [ ]:
def find_most_anterior_lung_points(lung_mask=None):
    # Check if the image is correctly oriented e.g. the sternum at the top, vertebrea at the bottom of the image,
    
    if 'lung_mask' in vars():
        lung_mask_90 = np.rot90(lung_mask)
        arr = np.nonzero(lung_mask_90)
        
        l1 = [arr[0][0]],[arr[1][0]]
        
        # Marker labelling to to able to distinct between masks
        ret, markers = cv2.connectedComponents(lung_mask_90)

        # één masker verwijderen
        markers[markers == markers[l1]] = 0 # markers in markers die ongelijk zijn aan de waarde van het eerste punt 
        arr2 = np.nonzero(markers)
        l2 = [arr2[0][0]],[arr2[1][0]] 
        # [arr2[as (0=rij of 1=kolom)][hoeveelste indices]],[arr2[as (0=rij of 1=kolom)][hoeveelste indices]] !!!
        
        # for later implementation:
        # if not fill in 1: for 90 degree rotation, 2: for 180 or 3: 270 degree rotation.
    else: 
        lung_mask_90 = None
        l1 = None
        l2 = None

    return lung_mask_90, l1, l2

# lung_mask_90, l1, l2 = find_most_anterior_lung_points()


In [ ]:
a = [5,5]

In [ ]:
b = np.zeros([10,10])

In [ ]:
imgplot = plt.imshow(b,cmap = 'gray')
plt.plot(a[0],a[1], marker='o', markersize=5, color="green")

Find sternum in sagittal plane
approach: search in the middle part of the image fot the longes bony structure (excluding the spine)

In [ ]:

# HUbone = 200 #bone is around 300-400 HU (taking artefacts such as streaking- and cupping artefacts in account so decreasing to 200)
# ret,thresh = cv2.threshold(sagittal,HUbone,3000,cv2.THRESH_BINARY)

# thres = thresh.astype('int16')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt

# bones_thresh_filt5 = cv2.medianBlur(thres, 5)
# bones_thresh_filt3 = cv2.medianBlur(thres, 3)
# fig, ax = plt.subplots(1, 2, figsize=(20, 20))
# ax[0].imshow(bones_thresh_filt3, cmap='gray')
# ax[0].set_title('median blur = 3')
# ax[1].imshow(bones_thresh_filt5, cmap='gray')
# ax[1].set_title('median blur = 5')

In [ ]:
# adj = np.zeros(sagittal.shape)
# adj[:,220:500]=1
# adj_im = bones_thresh_filt5*adj

# #closing uitvoeren
# kernel = np.ones((5,5),np.uint8)
# closing = cv2.morphologyEx(adj_im, cv2.MORPH_CLOSE, kernel, iterations=3)
# closing = closing.astype('int8')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt


# imgplot = plt.imshow(closing, cmap='gray')
# # Choose 4 or 8 for connectivity type
# connectivity = 4  
# masks = cv2.connectedComponentsWithStats(closing, connectivity, cv2.CV_32S)
# stats = masks[2]
# centroids = masks[3]
# areas = stats[:,4]
# areas.sort()
# sternum = areas[-2]

# a = areas == sternum
# b=[i for i, x in enumerate(a) if x]
# centroid_sternum = np.around(centroids[a])

# # dit geeft als het goed is een indicatie van waar het sternum ongeveer moet liggen in het axiale vlak (in dit geval rond de [322 342])

In [ ]:
def filtered_images(ct_image_2d):
    
    HUbone = 200 #bone is around 300-400 HU (taking artefacts such as streaking- and cupping artefacts in account so decreasing to 200)
    ret,thresh = cv2.threshold(ct_image_2d,HUbone,3000,cv2.THRESH_BINARY)
    
    # om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt
    thres = thresh.astype('int16')
    bone_threshold_filtered_image = cv2.medianBlur(thres, 3)
    
    return bone_threshold_filtered_image

def centroid_sternum(lung_mask_90, bone_threshold_filtered_image, l1, l2):
    # set values of columns before the first point and points after column of the second point to zero (of rotated original image)
    fov_between_lungs = l1[1], l2[1]
    fov_between_lungs = np.sort(fov_between_lungs, axis=None)
    rows = np.prod(lung_mask_90[1].shape)
    bone_threshold_filtered_image_90 = np.rot90(bone_threshold_filtered_image)
    mask = np.zeros(lung_mask_90.shape)
    mask[l1[0][0]-150:l1[0][0]+150,fov_between_lungs[0]:fov_between_lungs[1]]=1

    middle_col = np.around((fov_between_lungs[0]+fov_between_lungs[1])/2)
    # middle_row = np.around()
    print(fov_between_lungs[0],fov_between_lungs[1])
    fov_im90 = bone_threshold_filtered_image_90*mask
    
    
   
    # posterior middelpunt sternum bepalen:

    #closing uitvoeren
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(fov_im90, cv2.MORPH_CLOSE, kernel, iterations=3)
    closing = closing.astype('int8')# om te voorkomen dat je een of andere vage error over orde van grootte van de getallen krijgt
    


    # Choose 4 or 8 for connectivity type
    connectivity = 4  
    masks = cv2.connectedComponentsWithStats(closing, connectivity, cv2.CV_32S)
    output = masks[1]
    num_labels = masks[0]
    stats = masks[2]
    centroids = masks[3]

    for label in range(1,num_labels):
        blob_area = stats[label, cv2.CC_STAT_AREA]
        blob_width = stats[label, cv2.CC_STAT_WIDTH]
        blob_height = stats[label, cv2.CC_STAT_HEIGHT]

    imgplot = plt.imshow(fov_im90,cmap = 'gray')
    mask_area = stats[:,4]
    filtered_mask_area = np.sort([x for x in mask_area if x <= 100000 and x > 150])
    # imgplot = plt.imshow(filtered_mask_area, cmap='gray')
    a = stats[:,4] == filtered_mask_area[0]
    b=[i for i, x in enumerate(a) if x]
    centroid_sternum = np.around(centroids[b])
    
    
    row_centroid_sternum = np.uint(centroid_sternum[0][1])
    col_centroid_sternum = np.uint(centroid_sternum[0][0])
    centroid_axis = closing[:,col_centroid_sternum]
    centroid_axis_corr = centroid_axis[row_centroid_sternum:]
    # print(col_centroid_sternum)


    if col_centroid_sternum <= 412 and col_centroid_sternum >= 100:
        col_centroid_sternum = col_centroid_sternum
         # find indices where the value changes.
        index_backgroud_mask = np.where(centroid_axis_corr[:-1] != centroid_axis_corr[1:])[0] 
        # add the value of the first index to the centroid row value
        posterior_centroid_sternum_row = index_backgroud_mask[0]+row_centroid_sternum
    else: 
        col_centroid_sternum = 0
        posterior_centroid_sternum_row = 0
        # print('No valid sternum position could be found in this axial slice, LM2 is set to [0,0].')

   



    return posterior_centroid_sternum_row, col_centroid_sternum

In [ ]:
bone_ct = filtered_images(im_90)
sternum_indices = centroid_sternum(closing, bone_ct, l1, l2)